## **TRANSFORMACIÓN Y LIMPIEZA DE LOS DATOS:**

Continuamos el proyecto con la descarga de tres archivos en formato csv para proceder a su transformación y limpieza, relativos a:
- Número de llamadas recibidas por el 016
- Número de denuncias interpuestas.
- Número de víctimas mortales. 

No existen nulos ni duplicados, por lo que la limpieza se basa en transformar el tipo de dato de las columnas Año y Mes, cambiar el tipo en el número de llamadas, denuncias y víctimas a entero y cambiar el nombre de estas tres columnas en los tres archivos.
También eliminamos aquellos datos anteriores a 2020, para ceñirnos a aquellos años en que hubo campañas de prevención (entre 2020 y 2023) y creamos la columna de Trimestre para los casos en que no existe.

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

### **1. REGISTRO DE LLAMADAS RECIBIDAS POR EL 016:**

El Ministerio de Igualdad, por medio de la Delegación del Gobierno contra la Violencia de Género, presta el servicio telefónico de información, de asesoramiento y apoyo psicosocial para la atención de todas las formas de violencia contra las mujeres.

Tras la descarga de un archivo csv que contiene el registro de las llamadas recibidas según provincia, mes y año, se limpia el documento para obtener únicamente las llamadas recibidas entre 2020 y 2023, ya que estos son los años en que fueron publicadas las campañas de prevención.

Fuente: [Portal Estadístico Delegación del Gobierno contra la Violencia de Género](https://estadisticasviolenciagenero.igualdad.gob.es/)

In [2]:
llamadas = pd.read_csv('/Users/noeliarosonmartin/Ironhack/Project_ETL/data/data_original/llamadas.csv')
llamadas.head()

,Provincia,Año,Mes,Número de llamadas pertinentes
0,Almería,Año 2007,Septiembre,3.0
1,Almería,Año 2007,Octubre,52.0
2,Almería,Año 2007,Noviembre,57.0
3,Almería,Año 2007,Diciembre,69.0
4,Almería,Año 2008,Enero,91.0


In [ ]:
llamadas['Año'] = llamadas['Año'].str.replace(r'\D', '', regex=True) # Eliminamos lo que no es numérico para que tenga consistencia con las otras tablas

llamadas['Año'] = pd.to_datetime(llamadas['Año'], format='%Y').dt.year

In [4]:
llamadas = llamadas.loc[llamadas['Año'] >= 2020]   # Eliminamos los años anteriores a 2020
llamadas['Año'].unique()

array([2020, 2021, 2022, 2023], dtype=int32)

In [5]:
llamadas['Mes'].unique()

array(['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio',
       'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre'],
      dtype=object)

In [6]:
# Transformamos los meses a formato numérico para mantener la consistencia:

meses_año = {
    'Enero': '01',
    'Febrero': '02',
    'Marzo': '03',
    'Abril': '04',
    'Mayo': '05',
    'Junio': '06',
    'Julio': '07',
    'Agosto': '08',
    'Septiembre': '09',
    'Octubre': '10',
    'Noviembre': '11',
    'Diciembre': '12'}

llamadas['Mes'] = llamadas['Mes'].map(meses_año)

llamadas['Mes'] = llamadas['Mes'].astype(int)

In [7]:
llamadas.rename(columns={'Número de llamadas pertinentes': 'Numero_llamadas'}, inplace=True)
llamadas['Numero_llamadas'] = llamadas['Numero_llamadas'].astype(int)

In [ ]:
# Función creada para crear una nueva columna en la que aparezca el trimestre del año al que pertenece.
# Entra un mes y sale el número del trimestre

def trimestre(mes):
    if mes in [1, 2, 3]:
        return 1
    elif mes in [4, 5, 6]:
        return 2
    elif mes in [7, 8, 9]:
        return 3
    else:
        return 4

In [8]:
# Aplicamos la función trimestre:

llamadas['Trimestre'] = llamadas['Mes'].apply(trimestre)

In [9]:
# Reordenamos las columnas:

llamadas = llamadas [['Provincia', 'Año', 'Trimestre', 'Mes', 'Numero_llamadas']]

In [10]:
llamadas.duplicated().sum()

0

In [11]:
llamadas.isna().sum()

Provincia          0
Año                0
Trimestre          0
Mes                0
Numero_llamadas    0
dtype: int64

In [12]:
llamadas.info(memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
Index: 2331 entries, 148 to 10087
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Provincia        2331 non-null   object
 1   Año              2331 non-null   int32 
 2   Trimestre        2331 non-null   int64 
 3   Mes              2331 non-null   int64 
 4   Numero_llamadas  2331 non-null   int64 
dtypes: int32(1), int64(3), object(1)
memory usage: 247.9 KB


### **DATAFRAME FINAL *llamadas.csv***

In [13]:
llamadas = llamadas.reset_index(drop=True)
llamadas.head()

,Provincia,Año,Trimestre,Mes,Numero_llamadas
0,Almería,2020,1,1,68
1,Almería,2020,1,2,66
2,Almería,2020,1,3,75
3,Almería,2020,2,4,100
4,Almería,2020,2,5,79


In [14]:
# Exportamos csv:

llamadas.to_csv('../data/data_clean/llamadas_016.csv', index=False)

### **2. REGISTRO DE DENUNCIAS POR VIOLENCIA DE GÉNERO:**

El Ministerio de Igualdad publica el registro de denuncias en base a varios criterios, seleccionando en este caso para nuestro proyecto el número de denuncias según año, mes y provincia. Al igual que con la tabla anterior, se dejan los datos relativos a los años 2020 a 2023 y se limpian los datos de forma que mantenga la consistencia anterior.

Fuente: [Portal Estadístico Delegación del Gobierno contra la Violencia de Género](https://estadisticasviolenciagenero.igualdad.gob.es/)

In [ ]:
denuncias = pd.read_csv('/Users/noeliarosonmartin/Ironhack/Project_ETL/data/data_original/denunc.csv')
denuncias

,Provincia,Año,Trimestre,Número de denuncias por violencia de género
0,Almería,Año 2009,Primero,679.0
1,Almería,Año 2009,Segundo,715.0
2,Almería,Año 2009,Tercero,665.0
3,Almería,Año 2009,Cuarto,545.0
4,Almería,Año 2010,Primero,638.0
...,...,...,...,...
2959,Melilla,Año 2022,Primero,73.0
2960,Melilla,Año 2022,Segundo,94.0
2961,Melilla,Año 2022,Tercero,107.0
2962,Melilla,Año 2022,Cuarto,84.0


In [16]:
denuncias.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2964 entries, 0 to 2963
Data columns (total 4 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Provincia                                    2964 non-null   object 
 1   Año                                          2964 non-null   object 
 2   Trimestre                                    2964 non-null   object 
 3   Número de denuncias por violencia de género  2964 non-null   float64
dtypes: float64(1), object(3)
memory usage: 669.9 KB


In [17]:
denuncias['Año'] = denuncias['Año'].str.replace(r'\D', '', regex=True) # Eliminamos lo que no es numérico

denuncias['Año'] = pd.to_datetime(denuncias['Año'], format='%Y').dt.year  # Cambiamos a tipo datetime

In [18]:
denuncias = denuncias.loc[denuncias['Año'] >= 2020]  # Eliminamos los años anteriores a 2020

In [19]:
# Modificamos el formato de la columna Trimestre:

trimestr = {
    'Primero': 1,
    'Segundo': 2,
    'Tercero': 3,
    'Cuarto': 4}

denuncias['Trimestre'] = denuncias['Trimestre'].map(trimestr)


In [20]:
denuncias.rename(columns={'Número de denuncias por violencia de género': 'Numero_denuncias'}, inplace=True) # Renombramos la columna
denuncias['Numero_denuncias'] = denuncias['Numero_denuncias'].astype(int)  # Cambiamos a tipo entero


In [21]:
denuncias.duplicated().sum()

0

In [22]:
denuncias.isna().sum()

Provincia           0
Año                 0
Trimestre           0
Numero_denuncias    0
dtype: int64

In [23]:
denuncias.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 676 entries, 44 to 2963
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Provincia         676 non-null    object
 1   Año               676 non-null    int32 
 2   Trimestre         676 non-null    int64 
 3   Numero_denuncias  676 non-null    int64 
dtypes: int32(1), int64(2), object(1)
memory usage: 66.7 KB


### **DATAFRAME FINAL *denuncias.csv***

In [24]:
denuncias = denuncias.reset_index(drop=True)
denuncias.head()

,Provincia,Año,Trimestre,Numero_denuncias
0,Almería,2020,1,792
1,Almería,2020,2,678
2,Almería,2020,3,1109
3,Almería,2020,4,786
4,Almería,2021,1,630


In [25]:
# EXPORTAR CSV
denuncias.to_csv('../data/data_clean/denuncias_vg.csv', index=False)

### **3. REGISTRO DE VÍCTIMAS MORTALES POR VIOLENCIA DE GÉNERO:**

El Ministerio de Igualdad publica el registro de víctimas mortales en base a varios criterios, seleccionando en este caso para nuestro proyecto el número de víctimas según año, mes y provincia. Al igual que con la tabla anterior, se dejan los datos relativos a los años 2020 a 2023 y se limpian los datos de forma que mantenga la consistencia anterior.

Fuente: [Portal Estadístico Delegación del Gobierno contra la Violencia de Género](https://estadisticasviolenciagenero.igualdad.gob.es/)

In [26]:
victimas = pd.read_csv('/Users/noeliarosonmartin/Ironhack/Project_ETL/data/data_original/victimas_mort.csv')
victimas

,Año,Mes,Provincia,Número de mujeres víctimas mortales
0,Año 2003,Enero,Almería,1.0
1,Año 2003,Enero,Granada,1.0
2,Año 2003,Enero,Málaga,1.0
3,Año 2003,Enero,Santa Cruz de Tenerife,1.0
4,Año 2003,Enero,Barcelona,3.0
...,...,...,...,...
1127,Año 2023,Septiembre,Alicante/Alacant,1.0
1128,Año 2023,Septiembre,Castellón/Castelló,1.0
1129,Año 2023,Septiembre,Madrid,1.0
1130,Año 2023,Septiembre,Murcia,1.0


In [27]:
victimas.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1132 entries, 0 to 1131
Data columns (total 4 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Año                                  1132 non-null   object 
 1   Mes                                  1132 non-null   object 
 2   Provincia                            1132 non-null   object 
 3   Número de mujeres víctimas mortales  1132 non-null   float64
dtypes: float64(1), object(3)
memory usage: 257.3 KB


In [28]:
victimas['Año'] = victimas['Año'].str.replace(r'\D', '', regex=True) # Eliminamos lo que no es numérico

victimas['Año'] = pd.to_datetime(victimas['Año'], format='%Y').dt.year # Cambiamos a tipo datetime

In [29]:
victimas = victimas.loc[victimas['Año'] >= 2020]   # Eliminamos los años anteriores a 2020

In [30]:
victimas['Mes'] = victimas['Mes'].map(meses_año)   # Utilizamos el diccionario anterior para aplicar formato numérico a los meses

victimas['Mes'] = victimas['Mes'].astype(int) 

In [31]:
victimas['Trimestre'] = victimas['Mes'].apply(trimestre)  # Utilizamos la función anterior para crear la nueva columna de Trimestre


In [32]:
# Renombramos columna de número de víctimas y pasamos a tipo entero 

victimas.rename(columns={'Número de mujeres víctimas mortales': 'Num_victimas_mortales'}, inplace=True)
victimas['Num_victimas_mortales'] = victimas['Num_victimas_mortales'].astype(int)

In [33]:
# Reordenamos las columnas

victimas = victimas [['Provincia', 'Año', 'Trimestre', 'Mes', 'Num_victimas_mortales']]

In [34]:
victimas.isna().sum()

Provincia                0
Año                      0
Trimestre                0
Mes                      0
Num_victimas_mortales    0
dtype: int64

In [35]:
victimas.duplicated().sum()

0

In [36]:
victimas.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 188 entries, 944 to 1131
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Provincia              188 non-null    object
 1   Año                    188 non-null    int32 
 2   Trimestre              188 non-null    int64 
 3   Mes                    188 non-null    int64 
 4   Num_victimas_mortales  188 non-null    int64 
dtypes: int32(1), int64(3), object(1)
memory usage: 20.2 KB


### **DATAFRAME FINAL *victimas.csv***

In [37]:
victimas = victimas.reset_index(drop=True)
victimas.head()

,Provincia,Año,Trimestre,Mes,Num_victimas_mortales
0,Granada,2020,1,1,1
1,Ciudad Real,2020,1,1,1
2,Toledo,2020,1,1,1
3,Barcelona,2020,1,1,3
4,Lugo,2020,1,1,1


In [38]:
# EXPORTAR CSV
victimas.to_csv('../data/data_clean/victimas_mortales_vg.csv', index=False)